데이터 전처리
- 비속어와 비속어가 아닌 문장을 분류

In [1]:
import pandas as pd

# TSV 파일 경로
tsv_file_path = 'korean_unsmile_dataset/unsmile_train_v1.0.tsv'

# TSV 파일 열기
df = pd.read_csv(tsv_file_path, sep='\t')

# 데이터프레임 출력
df


,문장,여성/가족,남성,성소수자,인종/국적,연령,지역,종교,기타 혐오,악플/욕설,clean,개인지칭
0,일안하는 시간은 쉬고싶어서 그런게 아닐까,0,0,0,0,0,0,0,0,0,1,0
1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,0,0,0,0,0,0,1,0,0,0,0
2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,0,0,0,0,0,0,0,0,0,1,0
3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,0,0,0,0,0,0,0,0,0,1,0
4,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
15000,께롱께롱께롱!!!,0,0,0,0,0,0,0,0,0,1,0
15001,갱상도가아니라 홍어지 개좃같은 홍어년들,0,0,0,0,0,1,0,0,0,0,0
15002,말레이시아랑 인도네시아 여자 존나 못생겼던데,1,0,0,1,0,0,0,0,0,0,0
15003,링크도 안박고 가서 글 쓰자고 선동하네,0,0,0,0,0,0,0,0,1,0,0


In [2]:
df2 = pd.concat([df['문장'], df['clean']], axis=1)

In [3]:
df2

,문장,clean
0,일안하는 시간은 쉬고싶어서 그런게 아닐까,1
1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,0
2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,1
3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,1
4,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,0
...,...,...
15000,께롱께롱께롱!!!,1
15001,갱상도가아니라 홍어지 개좃같은 홍어년들,0
15002,말레이시아랑 인도네시아 여자 존나 못생겼던데,0
15003,링크도 안박고 가서 글 쓰자고 선동하네,0


In [24]:
df2.to_csv('clean_or_dirty_words.csv', encoding='utf-8')

Pretrained BERT 사용

In [30]:
!pip install transformers

     ---------------------------------------- 0.0/134.8 kB ? eta -:--:--
     -------- ------------------------------ 30.7/134.8 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 134.8/134.8 kB 2.0 MB/s eta 0:00:00
  Using cached PyYAML-6.0.1-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.8 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/8.8 MB 32.7 MB/s eta 0:00:01
   ---------- ----------------------------- 2.2/8.8 MB 23.8 MB/s eta 0:00:01
   ---------------- ----------------------- 3.6/8.8 MB 26.0 MB/s eta 0:00:01
   --------------------- ------------------ 4.8/8.8 MB 25.4 MB/s eta 0:00:01
   ------------------------- -------------- 5.6/8.8 MB 22.2 MB/s eta 0:00:01
   --------------------------- ------------ 6.1/8.8 MB 22.9 MB/s eta 0:00:01
   ---------

In [33]:
from transformers import TextClassificationPipeline, BertForSequenceClassification, AutoTokenizer
model_name = 'sgunderscore/hatescore-korean-hate-speech'
model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
pipe = TextClassificationPipeline(
        model = model,
        tokenizer = tokenizer,
        device = -1, # gpu: 0
        return_all_scores = True,
        function_to_apply = 'sigmoid')

c:\Users\bluecom014\miniconda3\envs\sesac\Lib\site-packages\transformers\pipelines\text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [27]:
#text = "야 이 병신같은 새끼야. 대가리를 왜 달고 다님?"
text = '안녕하세요'
label_and_score = pipe(text)[0]

label_and_score = sorted(label_and_score, key=lambda x:-x['score'])

if label_and_score[0]['label'] == 'None':
  print('정상 댓글')

else:
  print('비속어 감지')
  print(f'{label_and_score[0]["label"]} 비하 글이므로 필터링 필요.')

정상 댓글


In [11]:
df2

,문장,clean
0,일안하는 시간은 쉬고싶어서 그런게 아닐까,1
1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,0
2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,1
3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,1
4,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,0
...,...,...
15000,께롱께롱께롱!!!,1
15001,갱상도가아니라 홍어지 개좃같은 홍어년들,0
15002,말레이시아랑 인도네시아 여자 존나 못생겼던데,0
15003,링크도 안박고 가서 글 쓰자고 선동하네,0


In [29]:
from sklearn.metrics import accuracy_score
from tqdm import tqdm

labels = df2['clean'].tolist()
predictions = []
for data in tqdm(df2['문장'].tolist()):
    result = pipe(data)[0]
    result = sorted(result, key=lambda x:-x['score'])
    predict_label = 1 if result[0]['label'] == 'None' else 0
    predictions.append(predict_label)

accuracy = accuracy_score(labels, predictions)
accuracy

  0%|          | 0/15005 [00:00<?, ?it/s]

100%|██████████| 15005/15005 [16:54<00:00, 14.79it/s]


0.852315894701766

0 비속어, 민감정보 둘 다 없음  false, false  
1 비속어만 있음  true false  
2 민감정보만 있음  false true  
3 비속어, 민감정보 둘 다 있음  true true  

In [52]:
import re

sample_text = """
개인 정보 보호를 위해 전화번호 010-1234-5678와 이메일 test@example.com,
그리고 주민등록번호 920101-1234567을 마스킹합니다.
"""

def pattern_match(text):
    email_pattern = r'\b([A-Za-z0-9._%+-]{2})([A-Za-z0-9._%+-]+)@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    phone_pattern = r'(\d{3})-(\d{4})-(\d{4})'
    resident_id_pattern = r'(\d{6})-?([1-4])(\d{6})'

    matches_email = re.findall(email_pattern, text)
    matches_phone = re.findall(phone_pattern, text)
    matches_id = re.findall(resident_id_pattern, text)
    
    return matches_email or matches_phone or matches_id

# if pattern_match(sample_text):
#     print("패턴 적용")
# else:
#     print("패턴 적용 x")

def bad_words_find(text):
    label_and_score = pipe(text)[0]
    label_and_score = sorted(label_and_score, key=lambda x:-x['score'])
    return False if label_and_score[0]['label'] == 'None' else True

# bad_words_find(sample_text)

if bad_words_find(sample_text):
    label = 3 if pattern_match(sample_text) else 1
else:
    label = 2 if pattern_match(sample_text) else 0

label

2

In [54]:
!pip install konlpy

   ---------------------------------------- 0.0/19.4 MB ? eta -:--:--
   ---------------------------------------- 0.1/19.4 MB 3.4 MB/s eta 0:00:06
   - -------------------------------------- 0.7/19.4 MB 10.4 MB/s eta 0:00:02
   --- ------------------------------------ 1.5/19.4 MB 12.1 MB/s eta 0:00:02
   ---- ----------------------------------- 2.2/19.4 MB 13.1 MB/s eta 0:00:02
   ---- ----------------------------------- 2.4/19.4 MB 13.8 MB/s eta 0:00:02
   ----- ---------------------------------- 2.7/19.4 MB 10.6 MB/s eta 0:00:02
   ------ --------------------------------- 3.2/19.4 MB 10.1 MB/s eta 0:00:02
   -------- ------------------------------- 4.1/19.4 MB 11.5 MB/s eta 0:00:02
   ----------- ---------------------------- 5.4/19.4 MB 13.2 MB/s eta 0:00:02
   ------------- -------------------------- 6.4/19.4 MB 14.6 MB/s eta 0:00:01
   ------------- -------------------------- 6.8/19.4 MB 14.0 MB/s eta 0:00:01
   ---------------- ----------------------- 7.9/19.4 MB 14.8 MB/s eta 0:0

In [9]:
# https://frhyme.github.io/python/python_korean_englished/
# 문장을 자음, 모음 나누기

# 초성 리스트. 00 ~ 18
CHOSUNG_LIST = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
# 중성 리스트. 00 ~ 20
JUNGSUNG_LIST = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
# 종성 리스트. 00 ~ 27 + 1(1개 없음)
JONGSUNG_LIST = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

def korean_to_be_englished(korean_word):
    r_lst = []
    for w in list(korean_word.strip()):
        ## 영어인 경우 구분해서 작성함. 
        if '가'<=w<='힣':
            ## 588개 마다 초성이 바뀜. 
            ch1 = (ord(w) - ord('가'))//588
            ## 중성은 총 28가지 종류
            ch2 = ((ord(w) - ord('가')) - (588*ch1)) // 28
            ch3 = (ord(w) - ord('가')) - (588*ch1) - 28*ch2
            r_lst.append([CHOSUNG_LIST[ch1], JUNGSUNG_LIST[ch2], JONGSUNG_LIST[ch3]])
        else:
            r_lst.append([w])
    return r_lst
    
korean_to_be_englished("취뽀하고 싶다. 나 좀 데려가줘~~")

[['ㅊ', 'ㅟ', ' '],
 ['ㅃ', 'ㅗ', ' '],
 ['ㅎ', 'ㅏ', ' '],
 ['ㄱ', 'ㅗ', ' '],
 [' '],
 ['ㅅ', 'ㅣ', 'ㅍ'],
 ['ㄷ', 'ㅏ', ' '],
 ['.'],
 [' '],
 ['ㄴ', 'ㅏ', ' '],
 [' '],
 ['ㅈ', 'ㅗ', 'ㅁ'],
 [' '],
 ['ㄷ', 'ㅔ', ' '],
 ['ㄹ', 'ㅕ', ' '],
 ['ㄱ', 'ㅏ', ' '],
 ['ㅈ', 'ㅝ', ' '],
 ['~'],
 ['~']]

https://scienceon.kisti.re.kr/commons/util/originalView.do?cn=CFKO201924664108493&oCn=NPAP13263940&dbt=CFKO&journal=NPRO00383455  


In [2]:
# 자음, 모음 카데고리 사전 만들기
NUMBER_LIST = list(map(str, [i for i in range(10)]))
ALPHABET_LIST = [chr(i) for i in range(ord('a'), ord('z')+1)]
SPECIAL_WORD_LIST = ['!', '"', '#', '$', '%', '&', '\'', '?','@', '*', '+',',','-','.', '/', '~', ':', '^', ' ']
ONEWORD_LIST = ['ㄳ', 'ㄵ', 'ㅄ', 'ㄺ']

word_vector_list = CHOSUNG_LIST+JUNGSUNG_LIST+JONGSUNG_LIST+NUMBER_LIST+ALPHABET_LIST+SPECIAL_WORD_LIST+ONEWORD_LIST
print(word_vector_list)

['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', 'ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ', ' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '?', '@', '*', '+', ',', '-', '.', '/', '~', ':', '^', ' ', 'ㄳ', 'ㄵ', 'ㅄ', 'ㄺ']


In [3]:
# 카데고리 사전의 각 데이터와 인덱스를 딕셔너리로 추출
# 자음일 때 예를 들어 'ㄱ': [0, 41]은 ㄱ이 초성으로 오면 0, 종성으로 오면 41
# ' '(공백) 이면 ' ': [40, 122]. 40은 글자에 종성이 없을 때. 122는 단어와 단어 사이 띄어쓰기

word_vector_idx_dict = {}

for idx, data in enumerate(word_vector_list):
    if not data in word_vector_idx_dict:
        word_vector_idx_dict[data] = [] 
    word_vector_idx_dict[data].append(idx)
print(word_vector_idx_dict)

{'ㄱ': [0, 41], 'ㄲ': [1, 42], 'ㄴ': [2, 44], 'ㄷ': [3, 47], 'ㄸ': [4], 'ㄹ': [5, 48], 'ㅁ': [6, 56], 'ㅂ': [7, 57], 'ㅃ': [8], 'ㅅ': [9, 59], 'ㅆ': [10, 60], 'ㅇ': [11, 61], 'ㅈ': [12, 62], 'ㅉ': [13], 'ㅊ': [14, 63], 'ㅋ': [15, 64], 'ㅌ': [16, 65], 'ㅍ': [17, 66], 'ㅎ': [18, 67], 'ㅏ': [19], 'ㅐ': [20], 'ㅑ': [21], 'ㅒ': [22], 'ㅓ': [23], 'ㅔ': [24], 'ㅕ': [25], 'ㅖ': [26], 'ㅗ': [27], 'ㅘ': [28], 'ㅙ': [29], 'ㅚ': [30], 'ㅛ': [31], 'ㅜ': [32], 'ㅝ': [33], 'ㅞ': [34], 'ㅟ': [35], 'ㅠ': [36], 'ㅡ': [37], 'ㅢ': [38], 'ㅣ': [39], ' ': [40, 122], 'ㄳ': [43, 123], 'ㄵ': [45, 124], 'ㄶ': [46], 'ㄺ': [49, 126], 'ㄻ': [50], 'ㄼ': [51], 'ㄽ': [52], 'ㄾ': [53], 'ㄿ': [54], 'ㅀ': [55], 'ㅄ': [58, 125], '0': [68], '1': [69], '2': [70], '3': [71], '4': [72], '5': [73], '6': [74], '7': [75], '8': [76], '9': [77], 'a': [78], 'b': [79], 'c': [80], 'd': [81], 'e': [82], 'f': [83], 'g': [84], 'h': [85], 'i': [86], 'j': [87], 'k': [88], 'l': [89], 'm': [90], 'n': [91], 'o': [92], 'p': [93], 'q': [94], 'r': [95], 's': [96], 't': [97], 'u': [98], 'v': [9

In [4]:
JONGSUNG_ONLY = sorted(list(set(JONGSUNG_LIST)-set(CHOSUNG_LIST)))[1:]
JONGSUNG_ONLY

['ㄳ', 'ㄵ', 'ㄶ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅄ']

In [42]:
# 입력한 문장의 자음, 모음 벡터화

input_vector = [[0 for _ in range(127)] for _ in range(200)]

consonants_and_vowels = korean_to_be_englished("취뽀하고 싶다. 나 좀 데려가줘~~")

cnt = 0  # input_vector의 index
for words in consonants_and_vowels:
    if len(words) == 3:  # 한글일 때 (초성, 중성, 종성)
        for idx in range(3):
            if idx == 0 or idx == 1:  # 초성, 중성
                print(words[idx], word_vector_idx_dict[words[idx]][0], cnt)
                input_vector[cnt][word_vector_idx_dict[words[idx]][0]]=1
            
            else:  # 종성
                if words[idx] == ' ' or words[idx] in JONGSUNG_ONLY:  # 종성이 없을 때, 자음 중 종성만 있을 때
                    print(words[idx], word_vector_idx_dict[words[idx]][0], cnt)
                    input_vector[cnt][word_vector_idx_dict[words[idx]][0]]=1
                else:  # 자음 중 초성과 종성 둘 다 있는 경우, 종성
                    print(words[idx], word_vector_idx_dict[words[idx]][1], cnt)
                    input_vector[cnt][word_vector_idx_dict[words[idx]][1]]=1
            cnt += 1
            
    else:  # 한글이 아닌 경우 (특수문자, 영어)
        if words[0] == ' ':  # 띄어쓰기인 경우
            print(words[0], word_vector_idx_dict[words[0]][1], cnt)
            input_vector[cnt][word_vector_idx_dict[words[0]][1]]=1
        else:  # 특수문자, 영어인 경우
            print(words[0], word_vector_idx_dict[words[0]][0], cnt)
            input_vector[cnt][word_vector_idx_dict[words[0]][0]]=1
        cnt += 1

ㅊ 14 0
ㅟ 35 1
  40 2
ㅃ 8 3
ㅗ 27 4
  40 5
ㅎ 18 6
ㅏ 19 7
  40 8
ㄱ 0 9
ㅗ 27 10
  40 11
  122 12
ㅅ 9 13
ㅣ 39 14
ㅍ 66 15
ㄷ 3 16
ㅏ 19 17
  40 18
. 117 19
  122 20
ㄴ 2 21
ㅏ 19 22
  40 23
  122 24
ㅈ 12 25
ㅗ 27 26
ㅁ 56 27
  122 28
ㄷ 3 29
ㅔ 24 30
  40 31
ㄹ 5 32
ㅕ 25 33
  40 34
ㄱ 0 35
ㅏ 19 36
  40 37
ㅈ 12 38
ㅝ 33 39
  40 40
~ 119 41
~ 119 42


In [ ]:
# 입력한 문장의 자음, 모음 벡터화 (간단하게)

input_vector = [[0 for _ in range(127)] for _ in range(200)]
consonants_and_vowels = korean_to_be_englished("취뽀하고 싶다. 나 좀 데려가줘~~")

cnt = 0  # input_vector의 index
for words in consonants_and_vowels:
    if len(words) == 3:  # 한글일 때 (초성, 중성, 종성)
        for idx in range(3):
            if idx == 0 or idx == 1 or (idx == 2 and (words[idx] == ' ' or words[idx] in JONGSUNG_ONLY)):
                """
                1. 초성
                2. 중성 
                3. 종성이 없을 때 
                4. 종성이 있을 때, 자음 중 종성만 있으면
                """
                input_vector[cnt][word_vector_idx_dict[words[idx]][0]]=1
            else:  # 자음 중 초성과 종성 둘 다 있는 경우, 종성
                input_vector[cnt][word_vector_idx_dict[words[idx]][1]]=1
            cnt += 1
            
    else:  # 한글이 아닌 경우 (특수문자, 영어)
        if words[0] == ' ':  # 띄어쓰기인 경우
            input_vector[cnt][word_vector_idx_dict[words[0]][1]]=1
        else:  # 특수문자, 영어인 경우
            input_vector[cnt][word_vector_idx_dict[words[0]][0]]=1
        cnt += 1

BERT 모델 구현

In [31]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim

c:\Users\bluecom014\miniconda3\envs\sesac\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
df = pd.read_csv('clean_or_dirty_words.csv')
df

,Unnamed: 0,문장,clean
0,0,일안하는 시간은 쉬고싶어서 그런게 아닐까,1
1,1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,0
2,2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,1
3,3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,1
4,4,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,0
...,...,...,...
15000,15000,께롱께롱께롱!!!,1
15001,15001,갱상도가아니라 홍어지 개좃같은 홍어년들,0
15002,15002,말레이시아랑 인도네시아 여자 존나 못생겼던데,0
15003,15003,링크도 안박고 가서 글 쓰자고 선동하네,0


In [33]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [34]:
df

,문장,clean
0,일안하는 시간은 쉬고싶어서 그런게 아닐까,1
1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,0
2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,1
3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,1
4,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,0
...,...,...
15000,께롱께롱께롱!!!,1
15001,갱상도가아니라 홍어지 개좃같은 홍어년들,0
15002,말레이시아랑 인도네시아 여자 존나 못생겼던데,0
15003,링크도 안박고 가서 글 쓰자고 선동하네,0


In [35]:
# Bert 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

c:\Users\bluecom014\miniconda3\envs\sesac\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\bluecom014\.cache\huggingface\hub\models--bert-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [36]:
# 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_len):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self, idx):
        sentence = str(self.sentences[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [37]:
# Bert 모델 정의
class BertClassifier(nn.Module):
    def __init__(self):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, 2)  # 이진 분류를 위한 출력 레이어

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        output = self.dropout(pooled_output)
        return self.fc(output)

In [38]:
# 하이퍼파라미터
MAX_LEN = 128
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
EPOCHS = 5

# 데이터셋 및 데이터로더 생성
dataset = CustomDataset(
    sentences=df['문장'].values,
    labels=df['clean'].values,
    tokenizer=tokenizer,
    max_len=MAX_LEN
)

In [39]:
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [40]:
# 모델 초기화
model = BertClassifier()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [41]:
# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [45]:
for data in train_loader:
    print(data['attention_mask'])

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1

In [53]:
# 모델 학습
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for data in train_loader:
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {running_loss}")

tensor([[   101,   9604,  26344,  ...,      0,      0,      0],
        [   101,   9041,  78136,  ...,      0,      0,      0],
        [   101,   8857, 119093,  ...,      0,      0,      0],
        ...,
        [   101,   8857, 119093,  ...,      0,      0,      0],
        [   101,   9638,  71439,  ...,      0,      0,      0],
        [   101,   9834,  79940,  ...,      0,      0,      0]])


TypeError: dropout(): argument 'input' (position 1) must be Tensor, not str

In [ ]:
# 학습된 모델을 저장
torch.save(model.state_dict(), 'bert_classifier.pth')

# 문장을 비속어인지 판별하는 함수
def detect_profanity(sentence):
    model.eval()
    inputs = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        max_length=MAX_LEN,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
    )
    with torch.no_grad():
        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask)
        _, predicted = torch.max(outputs, 1)
    return 'Profane' if predicted.item() == 1 else 'Clean'

# 예시 문장을 사용하여 비속어 판별
sentence_to_check = "이 문장은 비속어가 포함되어 있습니다."
result = detect_profanity(sentence_to_check)
print(f"입력 문장: {sentence_to_check}")
print(f"판별 결과: {result}")